In [ ]:
from arcgis.learn import *
from arcgis.raster import ImageryLayer
from arcgis.gis import GIS
from arcpy import *
#from arcpy.ia import *
#from arcpy.sa import *

#import arcpy
import pandas as pd
import requests

SourceTile = r"U:\Project\027-15_2018a_4BAND.TIF"
arcpy.CheckOutExtension("ImageAnalyst")

The aim of this project was to identify areas of mixed land cover on orthophotography 4-band raster data (RGB + near infrared) using new-generation deep learning implementations.
Deep learning is a type of machine learning with several layers of nonlinear processing which allow users to identify patterns, objects, and pixels through models. It is a significant improvement on previous Machine Learning systems since it does not require vast amounts of training samples produced by expert users. Several ArcGis Pro Deep Learning models support sparse training data with remarkable success. In this project, training data for our models was gathered in intervals of less than 30.

There are several different algorithms and deep learning models that can be employed for image classification.  This project is a tool to carry out image classification of land cover by: preprocessing target data, exporting training samples, training and evaluating deep learning models, and displaying results so a human operator can then choose the best fitting model.

The first stage of the project consists on preprocessing our raster data.  This is to ensure images will be processed successfully with DL models at a later stage.

# 1 - Preprocessing

The first step in our analysis is to convert our target file into a raster object.  Next, we will preprocess our image to enhance.  This will gurantee that the classification process produces the best possible results.

In [ ]:
Source_Raster = Raster(SourceTile)
Source_Raster

## Sharpening

The first preprocessing step is to smooth the target image using the convolution function, which enhances an image with a sharpening 5 X 5 filter.

In [ ]:
# 14 identifies the 5x5 filter required 

Smooth_Raster = arcpy.sa.Convolution(Source_Raster, 14)
Source_Raster.save()
Source_Raster

## Stretching
The next preprocessing stage consists on enhancing an image by changing properties such as brightness, contrast, and gamma. In this case, we use a Sigmoid stretch as it highlights moderate pixel values while maintaining sufficient contrast in the perimeter. The function used applies a sigmoidal function (an S-shaped curve).

In [ ]:
# "3" indicates the sigmoid strength level. The minimum value is 0, and the maximum value is 6.

Stretched_Raster = arcpy.sa.Stretch(Source_Raster, "Sigmoid", 0, None, None, None, True, None, None, None, None, 3)
Stretched_Raster

## Resampling

Resampling changes the spatial resolution of a raster dataset and sets rules for aggregating or interpolating values across the new pixel size.  In this case, we will resample the image according to our datase spatial resolution (40 cm) and we will use the "Nearest" resampling technique, which is suitable for land cover and discrete data.

In [ ]:
# The new resampled file will be saved in ProjectData.

Str_Output_Raster = r"U:\ProjectData\resampled_output.tif"
arcpy.management.Resample(Stretched_Raster, Str_Output_Raster, 0.4, "Nearest")
Resampled_Raster = Raster(Str_Output_Raster)